# Import all necessary libraries -

In [1]:
from splinter import Browser
from sqlalchemy import func, create_engine
from bs4 import BeautifulSoup as bs
import requests, re, time, pymysql, pandas as pd
pymysql.install_as_MySQLdb()

# Get location for scrape

In [2]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/atom.php'
reqs = requests.get(url)
print(reqs)

<Response [200]>


## Using Beautiful Soup to get what's on the page so that we can navigate

In [4]:
soup = bs(reqs.text, 'lxml')
#print(soup)

## Take the links from the li and place them into empty list with .geojson instead of .atom at the end of each input.

In [5]:
listoflinks = []
for link in soup.find_all('li'):
    listoflinks.append(link.a['href'][:-4] + 'geojson')

In [6]:
url = url[:27]
for i in range(len(listoflinks)):
    listoflinks[i] = url + listoflinks[i]
#print(listoflinks)

# Make Database connection - Connect to Database 

In [11]:
remote_db_endpoint = ''
remote_db_port = ''
remote_db_user = ''
remote_db_pwd = ''
remote_db_name = ''

In [12]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Turn the list into a DataFrame

In [13]:
sqldf = pd.DataFrame({'geojsonLoc': listoflinks})
sqldf.count()

geojsonLoc    20
dtype: int64

## Make sure that the if_exists is what you want it to be. 

In [14]:
sqldf.to_sql('geojsonLoc', con=conn, if_exists='replace', index=False)

In [15]:
query = """SELECT * FROM geojsonLoc"""
implementCheck = pd.read_sql(query, con=conn)
implementCheck.count()

geojsonLoc    20
dtype: int64